In [1]:
from urllib.request import urlopen
from urllib.request import Request
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

last_page = 1
rating = 85

# ps, pc, xbox
platform = 'pc'

fullUrl = "https://www.futbin.com/players?page=" + str(last_page) + "&player_rating=" + str(rating) + "-" + str(rating) + "&sort=" + platform + "_price&order=asc"

options = Options()
options.add_argument('headless')
options.add_argument("disable-gpu")

driver = webdriver.Chrome('chromedriver', options=options)
driver.set_window_size(800, 600)

# get html to url
driver.get(fullUrl)
driver.implicitly_wait(3)
driver.find_element_by_xpath('//*[@id="desk-nav"]/div/a[2]/i').click()
driver.implicitly_wait(3)
driver.find_element_by_xpath('//*[@id="slide-out"]/ul/li[2]/ul/li[2]/a').click()
driver.implicitly_wait(3)
# ps version
# driver.find_element_by_xpath('//*[@id="slide-out"]/ul/li[2]/ul/li[2]/div/ul/li[1]/a').click()
# pc version
driver.find_element_by_xpath('//*[@id="slide-out"]/ul/li[2]/ul/li[2]/div/ul/li[3]/a').click()
# xbox version
# driver.find_element_by_xpath('//*[@id="slide-out"]/ul/li[2]/ul/li[2]/div/ul/li[2]/a').click()
driver.implicitly_wait(3)
driver.set_window_size(1920, 1080)
driver.implicitly_wait(3)

req = driver.page_source
bsObject = BeautifulSoup(req, 'html.parser')

In [2]:
pages = []

# import last page num
page_list = bsObject.select('div.col-md-12 > nav > ul')
for lists in page_list:
    page_row = lists.select('li')
    for page_fields in page_row:
        page = page_fields.text
        pages.append(page.strip())

last_page = int(pages[-2])

In [3]:
names = []
clns = []
clubs = []
leagues = []
nations = []
positions = []
versions = []
prices = []

for i in range(1, len(pages)+1):
    fullUrl = "https://www.futbin.com/players?page=" + str(i) + "&player_rating=" + str(rating) + "-" + str(rating) + "&sort=" + platform + "_price&order=asc"
    driver.get(fullUrl)
    req = driver.page_source
    bsObject = BeautifulSoup(req, "html.parser")
    
    character_list = bsObject.select('#repTb > tbody > tr')
    for lists in character_list:
        # import names
        name_row = lists.select('td > div > div > a')
        for name_fields in name_row:
            name = name_fields.text
            names.append(name.strip())
            
        # import clubs, leagues, nations
        cln_row = lists.select('span.players_club_nation > a')
        for cln_fields in cln_row:
            cln = cln_fields['data-original-title']
            clns.append(cln.strip())
            
        # import position and price
        position_row = lists.select('td')
        i = 0
        for position_fields in position_row:
            position = position_fields.text
            if i == 2:
                positions.append(position.strip())
            elif i == 4:
                price = position.strip()
                if price[-1] == 'K':
                    price = (float(price[:-1]) * 1000)
                elif price[-1] == 'M':
                    price = (float(price[:-1]) * 100000)
                prices.append(int(price))
            i += 1
        
        # import version
        version_row = lists.select('td.mobile-hide-table-col')
        for version_fields in version_row:
            version = version_fields.text
            versions.append(version.strip())
            
# import rating
rates = [rating for _ in range(len(names))]

for i in range(int(len(clns) / 3)):
    clubs.append(clns[i*3])
    leagues.append(clns[i*3+1])
    nations.append(clns[i*3+2])

In [4]:
driver.close()

In [5]:
character_df = pd.DataFrame({'Name':names, 'Club':clubs, 'League':leagues, 'Nation':nations, 'Rating':rates, 'Position':positions, 'Version':versions, 'Price':prices})

In [6]:
import time

now = time.localtime()
character_df.to_csv("./" + "%04d%02d%02d_%02d_%02d_%02d_output"%(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec) + ".csv", index=False, encoding='utf-8-sig')

In [7]:
character_df.head()

,Name,Club,League,Nation,Rating,Position,Version,Price
0,Mauro Icardi,Inter,Argentina,Serie A TIM,85,ST,Normal,0
1,Wilfried Zaha,Crystal Palace,Côte d'Ivoire,Premier League,85,ST,Special,0
2,Lucas Vázquez,Real Madrid,Spain,LaLiga Santander,85,RW,Special,0
3,Corentin Tolisso,FC Bayern München,France,Bundesliga,85,CM,Special,0
4,Alex Oxlade-Chamberlain,Liverpool,England,Premier League,85,CM,Special,0


In [9]:
# none zero prices
character_df = character_df[character_df['Price'] != 0]

In [10]:
character_df.head()

,Name,Club,League,Nation,Rating,Position,Version,Price
7,Péter Gulácsi,RB Leipzig,Hungary,Bundesliga,85,GK,CL,8500
8,Lopes,Olympique Lyonnais,Portugal,Ligue 1 Conforama,85,GK,Normal,8700
9,Zlatan Ibrahimovic,LA Galaxy,Sweden,Major League Soccer,85,ST,Normal,8900
10,Romelu Lukaku,Inter,Belgium,Serie A TIM,85,ST,Normal,9000
11,Péter Gulácsi,RB Leipzig,Hungary,Bundesliga,85,GK,Normal,9000


In [11]:
character_df.tail()

,Name,Club,League,Nation,Rating,Position,Version,Price
121,Clarence Seedorf,Icons,Holland,Icons,85,CM,Icon,945000
122,Gianfranco Zola,Icons,Italy,Icons,85,CF,Icon,960000
123,Rio Ferdinand,Icons,England,Icons,85,CB,Icon,110000
124,Michael Essien,Icons,Ghana,Icons,85,CDM,Icon,112000
125,Laurent Blanc,Icons,France,Icons,85,CAM,Icon,115999
